<a href="https://colab.research.google.com/github/Cherjimenez/Lab1_EddII/blob/main/Lab1_EDDII.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from graphviz import Digraph
from IPython.display import Image, display
from typing import Any, Optional

In [6]:
class Node:

    def __init__(self, data: Any) -> None:
        self.data = data
        self.left: Optional["Node"] = None
        self.right: Optional["Node"] = None
        self.height: int = 1


class AVLTree:

    def __init__(self) -> None:
        self.root: Optional["Node"] = None

    def insert(self, root: Optional[Node], node: Node) -> Node:
        if not root:
            return node
        if node.data['Title'] < root.data['Title']:
            root.left = self.insert(root.left, node)
        else:
            root.right = self.insert(root.right, node)

        root.height = 1 + max(self.get_height(root.left), self.get_height(root.right))

        balance = self.get_balance(root)

        # Rotaciones
        if balance > 1 and node.data['Title'] < root.left.data['Title']:
            return self.right_rotate(root)
        if balance < -1 and node.data['Title'] > root.right.data['Title']:
            return self.left_rotate(root)
        if balance > 1 and node.data['Title'] > root.left.data['Title']:
            root.left = self.left_rotate(root.left)
            return self.right_rotate(root)
        if balance < -1 and node.data['Title'] < root.right.data['Title']:
            root.right = self.right_rotate(root.right)
            return self.left_rotate(root)

        return root

    def delete(self, root: Optional[Node], title: str) -> Optional[Node]:
        if not root:
            return root
        if title < root.data['Title']:
            root.left = self.delete(root.left, title)
        elif title > root.data['Title']:
            root.right = self.delete(root.right, title)
        else:
            if not root.left:
                return root.right
            elif not root.right:
                return root.left
            temp = self.min_value_node(root.right)
            root.data = temp.data
            root.right = self.delete(root.right, temp.data['Title'])

        if not root:
            return root

        root.height = 1 + max(self.get_height(root.left), self.get_height(root.right))
        balance = self.get_balance(root)

        if balance > 1 and self.get_balance(root.left) >= 0:
            return self.right_rotate(root)
        if balance > 1 and self.get_balance(root.left) < 0:
            root.left = self.left_rotate(root.left)
            return self.right_rotate(root)
        if balance < -1 and self.get_balance(root.right) <= 0:
            return self.left_rotate(root)
        if balance < -1 and self.get_balance(root.right) > 0:
            root.right = self.right_rotate(root.right)
            return self.left_rotate(root)

        return root

    # Buscamos el sucesor
    def min_value_node(self, node: Node) -> Node:
        current = node
        while current.left:
            current = current.left
        return current


    def left_rotate(self, z: Node) -> Node:
        y = z.right
        T2 = y.left
        y.left = z
        z.right = T2
        z.height = 1 + max(self.get_height(z.left), self.get_height(z.right))
        y.height = 1 + max(self.get_height(y.left), self.get_height(y.right))
        return y

    def right_rotate(self, z: Node) -> Node:
        y = z.left
        T3 = y.right
        y.right = z
        z.left = T3
        z.height = 1 + max(self.get_height(z.left), self.get_height(z.right))
        y.height = 1 + max(self.get_height(y.left), self.get_height(y.right))
        return y

    def get_height(self, root: Optional[Node]) -> int:
        if not root:
            return 0
        return root.height

    def get_balance(self, root: Optional[Node]) -> int:
        if not root:
            return 0
        return self.get_height(root.left) - self.get_height(root.right)

    # Buscamos el nodo
    def search(self, root: Optional[Node], title: str, dad: Optional[Node] = None, gp: Optional[Node] = None) -> Optional[Node]:
      uncle = None
      if not root:
          return None, dad, uncle, gp

      if root.data['Title'] == title:
          if gp:
              if gp.left == dad:
                  uncle = gp.right
              else:
                  uncle = gp.left
          return root, dad, uncle, gp
      if title < root.data['Title']:
          return self.search(root.left, title, root, dad)
      return self.search(root.right, title, root, dad)

    def search_criteria(self, root: Optional[Node], year: int, foreign_earnings_min: float) -> list:
        found_movies = []
        if root:
            if root.data['Year'] == year and root.data['Domestic Percent Earnings'] < root.data['Foreign Percent Earnings'] and root.data['Foreign Earnings'] >= foreign_earnings_min:
                found_movies.append(root)
            found_movies.extend(self.search_criteria(root.left, year, foreign_earnings_min))
            found_movies.extend(self.search_criteria(root.right, year, foreign_earnings_min))
        return found_movies

    # Método para el recorrido por niveles
    def getLevels(self, node):
        levels=[]
        self.getLevelsHelper(node, levels, 0)
        flatList=[item for sublist in levels for item in sublist]
        return levels

    # Método auxiliar para el recorrido por niveles
    def getLevelsHelper(self, node, levels, level):
        if node is not None:
            if len(levels) == level:
                levels.append([])
            levels[level].append(node.data['Title'])
            self.getLevelsHelper(node.left, levels, level+1)
            self.getLevelsHelper(node.right, levels, level+1)

    # Nos retorna el nivel de un nodo
    def get_level(self, root: Optional[Node], node: Optional[Node], level: int = 0) -> int:
        if not root:
            return -1
        if root == node:
            return level
        left_level = self.get_level(root.left, node, level + 1)
        if left_level != -1:
            return left_level
        return self.get_level(root.right, node, level + 1)

    def get_balance_factor(self, node: Optional[Node]) -> int:
        if node:
            return self.get_height(node.right) - self.get_height(node.left)
        return 0

def visualize_tree(root: Optional[Node], i: int) -> None:
    dot = Digraph()

    def add_nodes_edges(root: Optional[Node]) -> None:
        if root:
            label = (
                f'{root.data["Title"]}\n'
                f'Año: {root.data["Year"]}\n'
                f'Mundial: {root.data["Worldwide Earnings"]}\n'
                f'Nacional: {root.data["Domestic Earnings"]}\n'
                f'Internacional: {root.data["Foreign Earnings"]}\n'
                f'Nacional%: {root.data["Domestic Percent Earnings"]}\n'
                f'Internacional%: {root.data["Foreign Percent Earnings"]}'
            )
            dot.node(root.data['Title'], label)
            if root.left:
                dot.edge(root.data['Title'], root.left.data['Title'])
                add_nodes_edges(root.left)
            if root.right:
                dot.edge(root.data['Title'], root.right.data['Title'])
                add_nodes_edges(root.right)

    add_nodes_edges(root)
    file_name = f'avl_tree_{i}.png'
    dot.render(file_name, format='png', cleanup=True)
    display(Image(filename=f'{file_name}.png'))

# Clase del menú
class AVLTreeMenu:
    def __init__(self):
        self.avl = AVLTree()
        self.i = 0
        self.root = None

        try:
            self.df = pd.read_csv('dataset_movies.csv')

        except FileNotFoundError:
            print("Error: No se encontró el archivo 'dataset_movies.csv'. Asegúrate de que esté en el directorio correcto.")
            self.df = None

    # Obtener los datos por pelicula
    def get_movie_data(self, title: str) -> Optional[Node]:
        if self.df is not None:
            movie_row = self.df[self.df['Title'] == title]
            if not movie_row.empty:
                row = movie_row.iloc[0]
                data = {
                    'Title': row['Title'],
                    'Year': row['Year'],
                    'Worldwide Earnings': row['Worldwide Earnings'],
                    'Domestic Earnings': row['Domestic Earnings'],
                    'Foreign Earnings': row['Foreign Earnings'],
                    'Domestic Percent Earnings': row['Domestic Percent Earnings'],
                    'Foreign Percent Earnings': row['Foreign Percent Earnings']
                }
                return Node(data)
        return None

    # Metodo que imprime los datos
    def print_movie_details(self, node: Node) -> None:
        if node:
            print(f"Título: {node.data['Title']}")
            print(f"Año: {node.data['Year']}")
            print(f"Ingresos Mundiales: {node.data['Worldwide Earnings']}")
            print(f"Ingresos Nacionales: {node.data['Domestic Earnings']}")
            print(f"Ingresos Internacionales: {node.data['Foreign Earnings']}")
            print(f"Porcentaje Nacional: {node.data['Domestic Percent Earnings']}")
            print(f"Porcentaje Internacional: {node.data['Foreign Percent Earnings']}")
        else:
            print("Nodo no encontrado.")

    def menu(self):
      while True:
          print("\n--- AVL Tree Menu ---")
          print("1. Insertar nodo")
          print("2. Eliminar nodo")
          print("3. Imprimir recorrido por niveles")
          print("4. Buscar película")
          print("5. Buscar películas por criterios")
          print("6. Salir")
          try:
              choice = int(input("Ingresa tu elección: "))
          except ValueError:
              print("La elección debe ser un número entero.")
              continue

          if choice == 1:
              # Insertar nodo
              title = input("Título de la película: ")
              node, pad, uncle, gp = self.avl.search(self.root, title)
              if node:
                  print(f"Película '{title}' ya existe en el árbol.")
                  continue
              node = self.get_movie_data(title)
              if node:
                  self.root = self.avl.insert(self.root, node)
                  print(f"Película '{title}' insertada correctamente.")
                  # Mostrar el árbol después de insertar
                  visualize_tree(self.root, self.i)
                  self.i += 1
              else:
                  print(f"Película '{title}' no encontrada en el dataset.")
          elif choice == 2:
              # Eliminar nodo
              title = input("Título de la película a eliminar: ")
              node = self.avl.search(self.root, title)
              if node:
                  self.root = self.avl.delete(self.root, title)
                  visualize_tree(self.root, self.i)
                  self.i += 1
                  print(f"Película '{title}' eliminada correctamente.")
              else:
                  print(f"Película '{title}' no encontrada.")
          elif choice == 3:
              # Imprimir recorrido por niveles
              print("Recorrido por niveles:")
              levels = self.avl.getLevels(self.root)
              for level in levels:
                  print("Nivel", levels.index(level) + 1, ":", level)
          elif choice == 4:
              # Buscar película
              title = input("Título de la película a buscar: ")
              node, dad, uncle, gp = self.avl.search(self.root, title)
              if node:
                  movie_data = node.data
                  print(f"Película encontrada:")
                  print(f"Título: {movie_data['Title']}")
                  print(f"Año: {movie_data['Year']}")
                  print(f"Ingresos Mundiales: {movie_data['Worldwide Earnings']}")
                  print(f"Ingresos Nacionales: {movie_data['Domestic Earnings']}")
                  print(f"Ingresos Internacionales: {movie_data['Foreign Earnings']}")
                  print(f"Porcentaje Nacional: {movie_data['Domestic Percent Earnings']}")
                  print(f"Porcentaje Internacional: {movie_data['Foreign Percent Earnings']}")

                  while True:
                      print(" a. nivel del nodo ")
                      print(" b. factor de balanceo del nodo ")
                      print(" c. padre del nodo ")
                      print(" d. abuelo del nodo ")
                      print(" e. tío del nodo ")
                      print(" f. Volver al menú principal ")

                      op = input("Ingresa tu elección: ")
                      if op == 'a':
                          level = self.avl.get_level(self.root, node)
                          print(f"Nivel del nodo: {level}")
                      elif op == 'b':
                          balance = self.avl.get_balance_factor(node)
                          print(f"Factor de balanceo del nodo: {balance}")
                      elif op == 'c':
                          print("\nPadre del nodo:")
                          if dad:
                              self.print_movie_details(dad)
                          else:
                              print("El nodo no tiene padre.")
                      elif op == 'd':
                          print("\nAbuelo del nodo:")
                          if gp:
                              self.print_movie_details(gp)
                          else:
                              print("El nodo no tiene abuelo.")
                      elif op == 'e':
                          print("\nTío del nodo:")
                          if uncle:
                              self.print_movie_details(uncle)
                          else:
                              print("El nodo no tiene tío.")
                      elif op == 'f':
                          break
                      else:
                          print("Opción no válida. Inténtalo de nuevo.")
              else:
                  print("Película no encontrada.")
          elif choice == 5:
              try:
                  year = int(input("Año de estreno: "))
                  min_foreign_earnings = float(input("Ingresos internacionales mínimos: "))
              except ValueError:
                  print("Año de estreno debe ser un número entero y los ingresos internacionales deben ser un número decimal.")
                  continue

              found_movies = self.avl.search_criteria(self.root, year, min_foreign_earnings)
              if found_movies:
                  print("Películas encontradas:")
                  for movie in found_movies:
                      print(f"Título: {movie.data['Title']}")
                      print(f"Año: {movie.data['Year']}")
                      print(f"Ingresos Mundiales: {movie.data['Worldwide Earnings']}")
                      print(f"Ingresos Nacionales: {movie.data['Domestic Earnings']}")
                      print(f"Ingresos Internacionales: {movie.data['Foreign Earnings']}")
                      print(f"Porcentaje Nacional: {movie.data['Domestic Percent Earnings']}")
                      print(f"Porcentaje Internacional: {movie.data['Foreign Percent Earnings']}")

                  while True:
                      print(" a. Seleccionar una película ")
                      print(" b. Volver al menú principal ")
                      op = input("Ingresa tu elección: ")
                      if op == 'a':
                          title = input("Título de la película a buscar: ")
                          for movie in found_movies:

                            if movie.data['Title'] == title:
                              node, dad, uncle, gp = self.avl.search(self.root, title)
                              while True:
                                  print(" a. nivel del nodo ")
                                  print(" b. factor de balanceo del nodo ")
                                  print(" c. padre del nodo ")
                                  print(" d. abuelo del nodo ")
                                  print(" e. tío del nodo ")
                                  print(" f. Volver al menú principal ")

                                  op = input("Ingresa tu elección: ")

                                  if op == 'a':
                                      level = self.avl.get_level(self.root, node)
                                      print(f"Nivel del nodo: {level}")
                                  elif op == 'b':
                                      balance = self.avl.get_balance_factor(node)
                                      print(f"Factor de balanceo del nodo: {balance}")
                                  elif op == 'c':
                                      print("\nPadre del nodo:")
                                      if dad:
                                          self.print_movie_details(dad)
                                      else:
                                          print("El nodo no tiene padre.")
                                  elif op == 'd':
                                      print("\nAbuelo del nodo:")
                                      if gp:
                                          self.print_movie_details(gp)
                                      else:
                                          print("El nodo no tiene abuelo.")
                                  elif op == 'e':
                                      print("\nTío del nodo:")
                                      if uncle:
                                          self.print_movie_details(uncle)
                                      else:
                                          print("El nodo no tiene tío.")
                                  elif op == 'f':
                                      break
                                  else:
                                      print("Opción no válida. Inténtalo de nuevo.")

                            else:
                              print("La pelicula no cumple con los criterios. ")
                      elif op == 'b':
                          break
                      else:
                          print("Opción no válida. Inténtalo de nuevo.")
              else:
                  print("No se encontraron películas con los criterios especificados.")
          elif choice == 6:
              break
          else:
              print("Opción no válida. Inténtalo de nuevo.")

# Función principal
def main():
    menu = AVLTreeMenu()
    menu.menu()

# Ejecutar la función principal
if __name__ == "__main__":
    main()





--- AVL Tree Menu ---
1. Insertar nodo
2. Eliminar nodo
3. Imprimir recorrido por niveles
4. Buscar película
5. Buscar películas por criterios
6. Salir
Ingresa tu elección: 6
